In [11]:
import cv2
import numpy as np
import random
from scipy.ndimage import rotate
import os
import pandas as pd
import itertools
from tqdm import tqdm




In [12]:
cell_size = 32
img_size = cell_size * 3
output_dir = os.path.join(os.getcwd(), "ttt_dataset_clean_rotated")
os.makedirs(output_dir, exist_ok=True)

winning_lines = [
    [(0, 0), (0, 1), (0, 2)],
    [(1, 0), (1, 1), (1, 2)],
    [(2, 0), (2, 1), (2, 2)],
    [(0, 0), (1, 0), (2, 0)],
    [(0, 1), (1, 1), (2, 1)],
    [(0, 2), (1, 2), (2, 2)],
    [(0, 0), (1, 1), (2, 2)],
    [(0, 2), (1, 1), (2, 0)]
]

def check_winner(board, player):
    return any(all(board[i][j] == player for i, j in line) for line in winning_lines)

def is_valid_game_state(board):
    flat = [cell for row in board for cell in row]
    x_count = flat.count('X')
    o_count = flat.count('O')
    if not (x_count == o_count or x_count == o_count + 1):
        return False
    x_win = check_winner(board, 'X')
    o_win = check_winner(board, 'O')
    if x_win and o_win:
        return False
    if x_win and x_count != o_count + 1:
        return False
    if o_win and x_count != o_count:
        return False
    return True

def draw_board(state, rotation_angle=0):
    img = np.ones((img_size, img_size), dtype=np.uint8) * 255
    for i in range(1, 3):
        cv2.line(img, (i * cell_size, 0), (i * cell_size, img_size), 0, 2)
        cv2.line(img, (0, i * cell_size), (img_size, i * cell_size), 0, 2)
    for i in range(3):
        for j in range(3):
            center = (j * cell_size + cell_size // 2, i * cell_size + cell_size // 2)
            if state[i][j] == 'X':
                cv2.line(img, (center[0] - 8, center[1] - 8), (center[0] + 8, center[1] + 8), 0, 2)
                cv2.line(img, (center[0] - 8, center[1] + 8), (center[0] + 8, center[1] - 8), 0, 2)
            elif state[i][j] == 'O':
                cv2.circle(img, center, 10, 0, 2)
    if rotation_angle != 0:
        img = rotate(img, angle=rotation_angle, reshape=False, mode='constant', cval=255)
    return img

# Generate all valid boards and apply 90°-based rotations
symbols = ['X', 'O', '']
all_boards = list(itertools.product(symbols, repeat=9))
labels = []
img_id = 0
rotation_angles = [0, 90, 180, 270]

for board_tuple in tqdm(all_boards, desc="Generating board images"):
    board = [list(board_tuple[i*3:(i+1)*3]) for i in range(3)]
    if not is_valid_game_state(board):
        continue
    if check_winner(board, 'X'):
        label = 0
    elif check_winner(board, 'O'):
        label = 1
    else:
        label = 2
    for angle in rotation_angles:
        img = draw_board(board, rotation_angle=angle)
        filename = f"{img_id:05d}_rot{angle}.png"
        cv2.imwrite(os.path.join(output_dir, filename), img)
        labels.append({'filename': filename, 'label': label})
        img_id += 1

# Save labels to CSV
df = pd.DataFrame(labels)
df.to_csv(os.path.join(output_dir, "labels.csv"), index=False)

Generating board images: 100%|██████████| 19683/19683 [00:27<00:00, 724.35it/s] 


In [1]:
import pandas as pd

df = pd.read_csv("labels.csv")
print(df['label'].value_counts())


label
2    18144
0     2504
1     1264
Name: count, dtype: int64
